In [2]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors

In [3]:
df = pd.read_csv("homework_6.1.csv")
df.head(5)

,Unnamed: 0,Z,X,Y
0,0,0.548814,0,-0.823220
1,1,0.715189,1,0.842405
2,2,0.602763,1,0.898618
3,3,0.544883,0,-0.817325
4,4,0.423655,0,-0.635482


In [5]:
treated = df[df["X"]== 1].reset_index(drop=True)
untreated = df[df["X"]==0].reset_index(drop=True)

Z_treated = treated[["Z"]]
Z_untreated = untreated[["Z"]]

# Match each untreated to  the nearest treat
nn_treated = NearestNeighbors(n_neighbors=1).fit(Z_untreated)
dist_treated, idx_treated = nn_treated.kneighbors(Z_treated)

# Match each treated to  the nearest untreat
nn_untreated = NearestNeighbors(n_neighbors=1).fit(Z_treated)
dist_untreated, idx_untreated = nn_untreated.kneighbors(Z_untreated)

# Extract Y for match pairs
Y_treated = treated["Y"].values
Y_untreated = untreated["Y"].values
Y_matched_untreated = untreated.loc[idx_treated.flatten(), 'Y'].values
Y_matched_treated = treated.loc[idx_untreated.flatten(), 'Y'].values

ATE = np.mean (Y_treated - Y_matched_untreated)
ATT = ATE
ATU = np.mean(Y_matched_treated-Y_untreated)
MTE = np.max(Y_matched_treated - Y_untreated)

print(f"Average Treatment Effect (ATE): {ATE:.4f}")
print(f"Average Treatment Effect on the Treated (ATT): {ATT:.4f}")
print(f"Average Treatment Effect on the Untreated (ATU): {ATU:.4f}")
print(f"Marginal Treatment Effect (MTE): {MTE:.4f}")

Average Treatment Effect (ATE): 1.8464
Average Treatment Effect on the Treated (ATT): 1.8464
Average Treatment Effect on the Untreated (ATU): 1.5495
Marginal Treatment Effect (MTE): 2.1725
